In [6]:
import sys
import numpy as np

sys.path.append(f"./../")
from src import SpaceEfficientVQF

In [7]:
m = 2459063
#m = 7 * 11
vqf = SpaceEfficientVQF(m)

total_var = len(vqf.p_bits) + len(vqf.q_bits)
n_known_bits = len(vqf.known_bits)

In [8]:
from src.clause_utils import get_key_by_value

r_vals = []
superposition_tables = vqf.superposition_tables
for t in superposition_tables:
    c = get_key_by_value(vqf.table_clause_dict, t)
    display(c.clause)
    display(t.table)
    r = np.abs(t.calc_r())
    r_vals.append(r)
    #print(f"Compression Ratio: {t.calc_r()}")
    #print("---------------------------------------")

compression_ratio = sum(r_vals) + n_known_bits

p_3 - 2*q_1*q_2 + q_1 + q_2 + q_3 - 2*z_3_4 - 4*z_3_5

,q_2,q_1,p_3,q_3
0,0,0,0,0
1,1,1,0,0
2,0,0,1,1
3,0,1,0,1
4,0,1,1,0
5,1,0,0,1
6,1,0,1,0
7,1,1,1,1


p_18*q_10 + p_19*q_9 + p_20*q_8 + p_21*q_7

,p_19,p_18,q_10,p_20,p_21,q_7,q_8,q_9
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1,1
4,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
76,1,1,0,0,1,0,0,0
77,1,1,0,0,1,0,1,0
78,1,1,0,1,0,0,0,0
79,1,1,0,1,0,1,0,0


In [9]:
print(f'Considering the number: {m}')
print(f'The total number of variables: {total_var}')
print(f'The number of known bits: {n_known_bits}')
print(f"R1: {round(r_vals[0], 5)}")
print(f"R2: {round(r_vals[1], 5)}")
print(f'The Compression Ratio: {round(compression_ratio, 5)}')

Considering the number: 2459063
The total number of variables: 33
The number of known bits: 4
R1: 1.0
R2: 1.66015
The Compression Ratio: 6.66015
